In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.layers import *
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import pathlib
from sklearn.metrics import precision_recall_fscore_support
import torch
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(9, 7)})


# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
# Coral Ordinal Function and Custom metric defining here
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
MODEL_PATH = '../data/models/'
DATA_DIR = '../data/'


2023-12-07 15:35:23.408413: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-07 15:35:23.408470: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-07 15:35:23.409391: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-07 15:35:23.415683: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-07 15:35:24.236137: W tensorflow/compiler/tf2

In [2]:
# Not to use in ipynb file
# import argparse
# # Create the parser
# parser = argparse.ArgumentParser(description="Script for generating the deep ensemble hyperparameter results")
# # Add the --value argument
# parser.add_argument('--dv', type=int, help="Data version")
# # Parse the arguments
# args = parser.parse_args()
# # Asking the user for their name
# data_type = int(args.dv)

data_type = 4

In [3]:

version = 'stl_1'
log_version = 'stl_1'
EPOCHS = 200

DATA_DIR = '../data'

pd.set_option('display.max_columns', None)

In [4]:
data_version = ''
if data_type != 4:

    data_version = 'synthetic'

    path = f'../data/new_data/data{data_type}/'

    train_df = pd.read_csv(f'{path}/Data{data_type}_Train.csv')
    val_df = pd.read_csv(f'{path}/Data{data_type}_Val.csv')
    test_df = pd.read_csv(f'{path}/Data{data_type}_Test.csv')

    tensor_train_data = torch.Tensor(train_df.Input1).unsqueeze(1)
    tensor_train_label = torch.Tensor(train_df.Output)

    tensor_val_data = torch.Tensor(val_df.Input1).unsqueeze(1)
    tensor_val_label = torch.Tensor(val_df.Output)

    tensor_test_data = torch.Tensor(test_df.Input1).unsqueeze(1)
    tensor_test_label = torch.Tensor(test_df.Output)

    train_dataset = TensorDataset(tensor_train_data, tensor_train_label)
    val_dataset = TensorDataset(tensor_val_data, tensor_val_label)

else:
    data_version = 'household_power'
    path = f'../data/{data_version}/'

    # feature_x = pd.read_csv(f'{path}/feature_x.csv')
    train_x = np.load(f'{path}/train_x.npy')
    train_y = np.load(f'{path}/train_y.npy')

    val_x = np.load(f'{path}/val_x.npy')
    val_y = np.load(f'{path}/val_y.npy')

    test_x = np.load(f'{path}/test_x.npy')
    test_y = np.load(f'{path}/test_y.npy')

    tensor_train_data = torch.Tensor(train_x)
    tensor_train_label = torch.Tensor(train_y)

    tensor_val_data = torch.Tensor(val_x)
    tensor_val_label = torch.Tensor(val_y)

    tensor_test_data = torch.Tensor(test_x)
    tensor_test_label = torch.Tensor(test_y)

    train_dataset = TensorDataset(tensor_train_data, tensor_train_label)
    val_dataset = TensorDataset(tensor_val_data, tensor_val_label)

In [5]:
import torch
import torch.optim as optim
from ray import tune

class MCDO(nn.Module):
    def __init__(self, num_features, hidden_size, n_layers, drop_out):
        super(MCDO, self).__init__()

        layers = []
        input_size = num_features

        # Create 'n' linear layers
        for _ in range(n_layers - 1):  # n_layers - 1 because the last layer is the output layer
            layers.append(nn.Linear(input_size, hidden_size))
            input_size = hidden_size

        # The last layer outputs a single value
        self.out = nn.Linear(hidden_size, 1)
        self.layers = nn.ModuleList(layers)
        self.dropout = nn.Dropout(p=drop_out)

    def forward(self, x):
        for layer in self.layers:
            x = self.dropout(torch.relu(layer(x)))
        out = self.out(x)  # Apply the output layer
        return out



2023-12-07 15:35:27,134	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-12-07 15:35:27,209	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [6]:
models = {}

In [7]:
def train_mcdo(config):

    hidden_size = int(config['hidden_size'])
    num_layers = int(config['num_layers'])
    patience = int(config['patience'])
    batch_size = int(config['batch_size'])
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

    model_name = f'hs_{hidden_size}_nl_{num_layers}_do_{config["drop_out"]}_ep_{config["epochs"]}_p_{config["patience"]}_lr_{config["lr"]}_bs_{batch_size}.pt'

    models[f'{model_name}']= MCDO(config["num_features"], hidden_size, num_layers, config["drop_out"])

    criterion = torch.nn.MSELoss()
    optimizer = optim.Adam(models[f'{model_name}'].parameters(), lr=config["lr"])
    device = 'cpu'
    models[f'{model_name}'].to(device)
    
    best_model_dir = f'./models/'
    pathlib.Path(best_model_dir).mkdir(parents=True, exist_ok=True) 
    best_model_path = f'{best_model_dir}/hs_{hidden_size}_nl_{num_layers}_do_{config["drop_out"]}_ep_{config["epochs"]}_p_{config["patience"]}_lr_{config["lr"]}_bs_{batch_size}'
    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(config["epochs"]):
        models[f'{model_name}'].train()
        # Train loop
        for batch in train_loader:
            x = batch[0].to(device)
            y = batch[1].to(device)
            optimizer.zero_grad()
            out = models[f'{model_name}'](x)
            loss = criterion(y, out)
            loss.backward()
            optimizer.step()

        # Validation loop
        models[f'{model_name}'].eval()
        val_losses = []
        with torch.no_grad():
            for batch in val_loader:
                x = batch[0].to(device)
                y = batch[1].to(device)
                out = models[f'{model_name}'](x)
                val_loss = criterion(y, out)
                val_losses.append(val_loss.item())
        avg_val_loss = sum(val_losses) / len(val_losses)

        # Early stopping and model saving logic
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
            # Save the model state
            torch.save(models[f'{model_name}'].state_dict(), best_model_path)
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f'Early stopping triggered at epoch {epoch}')
                break


    tune.report(train_loss=loss,best_val_loss=best_val_loss)

In [8]:


def main(num_samples=100, max_num_epochs=500):
    ray.init()

    if data_type != 4:
        num_features = 1
    else:
        num_features = 7

    config = {
        "num_features": num_features,
        "hidden_size": tune.quniform(1, 200, 1),
        "num_layers": tune.quniform(2, 10, 1),
        "drop_out": tune.uniform(0.1, 0.5),
        "lr": tune.loguniform(1e-4, 1e-1),
        "epochs": max_num_epochs,
        "patience": tune.quniform(1, 50, 1),
        "batch_size": tune.quniform(1, 20, 1)
    }

    from ray.tune.search import ConcurrencyLimiter
    from ray.tune.search.bayesopt import BayesOptSearch
    algo = BayesOptSearch(utility_kwargs={"kind": "ucb", "kappa": 2.5, "xi": 0.0})
    algo = ConcurrencyLimiter(algo, max_concurrent=50)

    result = tune.run(
            tune.with_parameters(train_mcdo),
            config=config,
            resources_per_trial={"cpu": 1},
            metric="best_val_loss",
            mode="min",
            search_alg=algo,
            num_samples=num_samples,
            # max_concurrent_trials=20,
            local_dir= f'/home/sgupta/WORK/Triplevel_transformer_model/baselines/hyperparameter_search',
            name=f"experiment_mcdo_data{data_type}",
            max_failures=7,
            raise_on_failed_trial=False
        )

    temp = result.dataframe()   
    hyperparams_path = f'./hyperparameter_search/'
    temp.to_csv(f"{hyperparams_path}/RAY_RESULTS_mcdo_data{data_type}.csv")


In [9]:
import ray

main(500,500)
#  Stop Ray
ray.shutdown()

2023-12-07 15:35:29,093	INFO worker.py:1621 -- Started a local Ray instance.
2023-12-07 15:35:30,524	INFO tune.py:657 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
2023-12-07 15:35:30,546	WARNING bayesopt_search.py:421 -- BayesOpt search does not support quantization. Dropped quantization.
2023-12-07 15:35:30,546	WARNING bayesopt_search.py:431 -- BayesOpt does not support specific sampling methods. The Uniform sampler will be dropped.
2023-12-07 15:35:30,547	WARNING bayesopt_search.py:421 -- BayesOpt search does not support quantization. Dropped quantization.
2023-12-07 15:35:30,547	WARNING bayesopt_search.py:431 -- BayesOpt does not support specific sampling methods. The Uniform sampler will be dropped.
2023-12-07 15:35:30,548	WARNING bayesopt_search.py:431 -- BayesOpt does not support specific sampling methods. The LogUniform

(train_mcdo pid=97940) /media/seconddrive/sgupta/anaconda3/envs/py39SAM/lib/python3.9/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
(train_mcdo pid=97940)   return F.mse_loss(input, target, reduction=self.reduction)
(train_mcdo pid=97940) /media/seconddrive/sgupta/anaconda3/envs/py39SAM/lib/python3.9/site-packages/torch/autograd/__init__.py:251: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11070). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
(train_mcdo pid=97

Trial name,best_val_loss,train_loss
train_mcdo_31af555f,0.0168462,0.0610774
train_mcdo_332d9fd0,0.0168462,2.89229e-05
train_mcdo_36e2d329,0.0168698,0.0603368
train_mcdo_4d96575e,0.0168461,0.0775159
train_mcdo_61d93178,0.0168471,0.000415991
train_mcdo_7695fbae,0.0168461,0.0194314
train_mcdo_80625ad5,0.016846,0.004257
train_mcdo_93e179be,0.016846,0.0339997
train_mcdo_cefd118d,0.0168461,0.0354984


(train_mcdo pid=97940) Early stopping triggered at epoch 9


(raylet) [2023-12-07 15:39:49,347 E 97152 97164] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-12-07_15-35-27_353429_96939 is over 95% full, available space: 42519707648; capacity: 1006450962432. Object creation will fail if spilling is required.
(raylet) [2023-12-07 15:39:59,359 E 97152 97164] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-12-07_15-35-27_353429_96939 is over 95% full, available space: 42519273472; capacity: 1006450962432. Object creation will fail if spilling is required.
(raylet) [2023-12-07 15:40:09,371 E 97152 97164] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-12-07_15-35-27_353429_96939 is over 95% full, available space: 42518839296; capacity: 1006450962432. Object creation will fail if spilling is required.
(raylet) [2023-12-07 15:40:19,384 E 97152 97164] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-12-07_15-35-27_353429_96939 is over 95% full, available space: 42518409216; capacity: 1006450962432. Object crea

...

(raylet) [2023-12-07 16:06:00,830 E 97152 97164] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-12-07_15-35-27_353429_96939 is over 95% full, available space: 42529173504; capacity: 1006450962432. Object creation will fail if spilling is required.
(raylet) [2023-12-07 16:06:10,838 E 97152 97164] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-12-07_15-35-27_353429_96939 is over 95% full, available space: 42528739328; capacity: 1006450962432. Object creation will fail if spilling is required.
(raylet) [2023-12-07 16:06:20,847 E 97152 97164] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-12-07_15-35-27_353429_96939 is over 95% full, available space: 42528309248; capacity: 1006450962432. Object creation will fail if spilling is required.
(raylet) [2023-12-07 16:06:30,859 E 97152 97164] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-12-07_15-35-27_353429_96939 is over 95% full, available space: 42527838208; capacity: 1006450962432. Object crea

(train_mcdo pid=98396) Early stopping triggered at epoch 104


(raylet) [2023-12-07 16:15:41,445 E 97152 97164] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-12-07_15-35-27_353429_96939 is over 95% full, available space: 42503348224; capacity: 1006450962432. Object creation will fail if spilling is required.
(raylet) [2023-12-07 16:15:51,452 E 97152 97164] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-12-07_15-35-27_353429_96939 is over 95% full, available space: 42502905856; capacity: 1006450962432. Object creation will fail if spilling is required.
(raylet) [2023-12-07 16:16:01,460 E 97152 97164] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-12-07_15-35-27_353429_96939 is over 95% full, available space: 42502471680; capacity: 1006450962432. Object creation will fail if spilling is required.
(raylet) [2023-12-07 16:16:11,474 E 97152 97164] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-12-07_15-35-27_353429_96939 is over 95% full, available space: 42502037504; capacity: 1006450962432. Object crea

(train_mcdo pid=98237) Early stopping triggered at epoch 89


(raylet) [2023-12-07 16:30:22,284 E 97152 97164] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-12-07_15-35-27_353429_96939 is over 95% full, available space: 42504298496; capacity: 1006450962432. Object creation will fail if spilling is required.
(raylet) [2023-12-07 16:30:32,292 E 97152 97164] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-12-07_15-35-27_353429_96939 is over 95% full, available space: 42503778304; capacity: 1006450962432. Object creation will fail if spilling is required.
(raylet) [2023-12-07 16:30:42,300 E 97152 97164] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-12-07_15-35-27_353429_96939 is over 95% full, available space: 42503356416; capacity: 1006450962432. Object creation will fail if spilling is required.
(raylet) [2023-12-07 16:30:52,307 E 97152 97164] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-12-07_15-35-27_353429_96939 is over 95% full, available space: 42502926336; capacity: 1006450962432. Object crea